<a href="https://colab.research.google.com/github/kairosdumble/HeartBeatCheck/blob/main/ECG_AI_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch
!pip install numpy
!pip install scikit-learn

In [ ]:
!pip -q install wfdb numpy pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 108.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.2 which is incompatible.
dask-cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.2 which is incompatible.
cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.2 which is incompatible.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import wfdb, numpy as np, pandas as pd

from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from pathlib import Path
from scipy.signal import butter, lfilter

wfdb.dl_database('mitdb', dl_dir='mitdb')
BASE = Path('mitdb')

Generating record list for: 100
Generating record list for: 101
Generating record list for: 102
Generating record list for: 103
Generating record list for: 104
Generating record list for: 105
Generating record list for: 106
Generating record list for: 107
Generating record list for: 108
Generating record list for: 109
Generating record list for: 111
Generating record list for: 112
Generating record list for: 113
Generating record list for: 114
Generating record list for: 115
Generating record list for: 116
Generating record list for: 117
Generating record list for: 118
Generating record list for: 119
Generating record list for: 121
Generating record list for: 122
Generating record list for: 123
Generating record list for: 124
Generating record list for: 200
Generating record list for: 201
Generating record list for: 202
Generating record list for: 203
Generating record list for: 205
Generating record list for: 207
Generating record list for: 208
Generating record list for: 209
Generati

In [ ]:
CUTOFF = 0.5  # Hz
FS = 360
SAMPLING_RATE =360
WINDOW_SIZE = 361

# 1.1 학습용/ 검증용/ 평가용데이터 분리

In [ ]:
#DS1/DS2(환자 분리)
DS1 = [101,106,108,109,112,114,115,116,118,119,122,124,201,203,205,207,208,209,215,220,223,230]
DS2 = [100,103,105,111,113,117,121,123,200,202,210,212,213,214,219,221,222,228,231,232,233,234]

#숫자 리스트를 문자열 리스트로 변환
DS1 = list(map(str, DS1))
DS2 = list(map(str, DS2))

VAL_N = 4 # DS1의 마지막 4개를 검증용으로 사용
train_recs = DS1[:-VAL_N] # 학습용 데이터 (모델이 가중치를 배우는 데이터)
val_recs   = DS1[-VAL_N:] # 검증용 데이터(학습 중 과적합 여부 확인 / 하이퍼파라미터 튜닝)
test_recs  = DS2          # 평가용 데이터(모델이 전혀 보지 못한 환자 데이터로)

print('Train:', train_recs)
print('Val  :', val_recs)
print('Test :', test_recs)


Train: ['101', '106', '108', '109', '112', '114', '115', '116', '118', '119', '122', '124', '201', '203', '205', '207', '208', '209']
Val  : ['215', '220', '223', '230']
Test : ['100', '103', '105', '111', '113', '117', '121', '123', '200', '202', '210', '212', '213', '214', '219', '221', '222', '228', '231', '232', '233', '234']


In [ ]:
print(len(train_recs))
print(len(val_recs))
print(len(test_recs))

# 1.2 AAMI 기준을 이진 분류(정상 vs 부정맥)

In [ ]:
#AAMI 매핑 & 제외 규칙
AAMI_MAP = {
    'N': set(['N','L','R','e','j']),
    'S': set(['A','a','J','S']),
    'V': set(['V','E']),
    'F': set(['F']),
    'Q': set(['Q'])
}
EXCLUDE = set(['/','f','Q','|','x','~','!','[',']','+', '"'])

# ECG 신호의 어노테이션(annotation)을 5단계로 축소
def aami_class(sym): # sym :MIT-BIH beat, cls : AAMI_5단계 분류
    for cls, syms in AAMI_MAP.items():
        if sym in syms: return cls
    return None

# aami_class의 5단계 구조 ->이진 분류
def bin_label_from_aami(aami_cls): # aami_cls: aami_class에서 나온결과(5가지 중 1개)
    if aami_cls == 'N': return 0  # 정상
    if aami_cls in ('S','V','F'): return 1  # 부정맥
    return None  # Q 등 제외


# 이진 라벨 부여

In [ ]:
#레코드당 어노테이션(의사(전문가)가 붙인 이벤트 정보) 읽고 이진 라벨 부여
def load_record_beats(rec_id:str):
    ann = wfdb.rdann(str(BASE/rec_id), 'atr')
    df = pd.DataFrame({'sample': ann.sample, 'symbol': ann.symbol}) # sample,symbol열 추가

    # 제외( paced/논비트/분류불가 ) 제거
    df = df[~df['symbol'].isin(EXCLUDE)].copy()

    # AAMI 클래스 찾기(모르는 심볼은 버림)
    df['aami'] = df['symbol'].apply(aami_class) #aami열 추가
    df = df[df['aami'].notnull()].copy()

    # 이진 라벨
    df['label'] = df['aami'].apply(bin_label_from_aami) #label열 추가
    df = df[df['label'].notnull()].copy()
    df['rec_id'] = rec_id # rec_id열 추가

    return df.reset_index(drop=True) # 행 인덱스를 0-N-1로 리셋해서 깔끔하게 반환


In [ ]:
# 예시
print(load_record_beats('100'))

      sample symbol aami  label rec_id
0         77      N    N      0    100
1        370      N    N      0    100
2        662      N    N      0    100
3        946      N    N      0    100
4       1231      N    N      0    100
...      ...    ...  ...    ...    ...
2268  648978      N    N      0    100
2269  649232      N    N      0    100
2270  649484      N    N      0    100
2271  649734      N    N      0    100
2272  649991      N    N      0    100

[2273 rows x 5 columns]


In [ ]:
# 학습/검증/테스트 데이터 별로 나눠서 합치기
def build_split(recs): # recs : 레코드 ID들의 리스트
    parts = []
    for rec_id in recs:
        try:
            parts.append(load_record_beats(rec_id)) # 이진분류 결과를 저장
        except Exception as e:
            print(f'[WARN] {rec_id}: {e}')
    return pd.concat(parts, ignore_index=True) if parts else pd.DataFrame()

In [ ]:
print(build_split(test_recs))

       sample symbol aami  label rec_id
0          77      N    N      0    100
1         370      N    N      0    100
2         662      N    N      0    100
3         946      N    N      0    100
4        1231      N    N      0    100
...       ...    ...  ...    ...    ...
49700  648797      N    N      0    234
49701  649040      N    N      0    234
49702  649292      N    N      0    234
49703  649536      N    N      0    234
49704  649772      N    N      0    234

[49705 rows x 5 columns]


In [ ]:
train_df = build_split(train_recs)
val_df = build_split(val_recs)
test_df = build_split(test_recs)

#학습을 위한 라벨 생성
train_labels = train_df['label'].to_numpy()
val_labels = val_df['label'].to_numpy()
test_labels = test_df['label'].to_numpy()

print('\n\n======================train_labels 측정====================')
print(train_labels)
print('\n\n======================val_labels 측정======================')
print(val_labels)
print('\n\n======================test_labels 측정====================')
print(test_labels)

for name, df in [('train', train_df), ('val', val_df), ('test', test_df)]: #train,val,test반복
    total = len(df)
    pos = int(df['label'].sum()) #label 컬럼의 합 (정상=0, 부정맥=1이라고 가정하니까, 합계를 구하면 곧 부정맥 개수)
    neg = total - pos # 전체 개수에서 부정맥 개수를 빼면 정상 개수
    print(f'{name:5s} -> total {total:6d} | 정상(0): {neg:6d} | 부정맥(1): {pos:6d}')



======================train_labels 측정====================
[0 0 0 ... 0 0 0]


======================val_labels 측정======================
[0 0 0 ... 0 0 0]


======================test_labels 측정====================
[0 0 0 ... 0 0 0]
train -> total  40741 | 정상(0):  36417 | 부정맥(1):   4324
val   -> total  10272 | 정상(0):   9449 | 부정맥(1):    823
test  -> total  49705 | 정상(0):  44259 | 부정맥(1):   5446


In [ ]:
# 차후 학습 샘플링을 위한 samples 정리
train_samples = {}
val_samples = {}
test_samples = {}

for rec_id in train_df['rec_id'].unique():
  train_samples[rec_id] = train_df[train_df['rec_id'] == rec_id]['sample'].to_numpy()

for rec_id in val_df['rec_id'].unique():
  val_samples[rec_id] = val_df[val_df['rec_id'] == rec_id]['sample'].to_numpy()

for rec_id in test_df['rec_id'].unique():
  test_samples[rec_id] = test_df[test_df['rec_id'] == rec_id]['sample'].to_numpy()

print('\n\n======================train_samples 측정====================')
print(train_samples)
print('\n\n======================val_samples 측정======================')
print(val_samples)
print('\n\n======================test_samples 측정====================')
print(test_samples)



======================train_samples 측정====================
{'101': array([    83,    396,    711, ..., 649004, 649372, 649751]), '106': array([   351,    724,   1086, ..., 649162, 649340, 649791]), '108': array([    88,    442,    789, ..., 649170, 649471, 649769]), '109': array([   111,    343,    571, ..., 649378, 649663, 649931]), '112': array([   124,    382,    644, ..., 649361, 649626, 649883]), '114': array([   310,    726,   1143, ..., 649261, 649523, 649783]), '115': array([   161,    518,    850, ..., 649357, 649647, 649955]), '116': array([   282,    561,    837, ..., 649442, 649701, 649957]), '118': array([    68,    369,    674, ..., 649226, 649429, 649742]), '119': array([   309,    503,    977, ..., 649129, 649468, 649788]), '122': array([    93,    326,    561, ..., 649383, 649643, 649905]), '124': array([   243,    693,   1119, ..., 649146, 649510, 649876]), '201': array([   159,    415,    686, ..., 648594, 649104, 649800]), '203': array([    99,    431,    660, ...

---

# 2.1 HPF 제거 & Baseline 제거 & 슬라이싱 & 정규화

fc = 0.5 Hz

MIT-BIH는 보통 fs=360 Hz이므로

wn = fc / (fs/2) 로 정규화

하이패스 필터(HPF) 적용

Butterworth 하이패스 필터를 적용

In [ ]:
def highpass_filter(data, order=5): #data는 np.ndarray형식
    nyquist = 0.5 * FS
    normal_cutoff = CUTOFF / nyquist
    b, a = butter(order, normal_cutoff, btype='high', analog=False) # order은 필터 차수
    # 위상왜곡 제거(오프라인 전처리)
    filtered_data = lfilter(b, a, data) #IIR/FIR 필터를 선형 차분 방정식(디지털 필터식)에 따라 적용
    return filtered_data

In [ ]:
from torch.utils.data import Dataset, DataLoader

class ECGDataset(Dataset):
    def __init__(self, df, base_path=BASE, window_size=WINDOW_SIZE):
        self.entries = df[['rec_id','sample','label']].to_records(index=False)
        self.base = base_path
        self.window_size = window_size
        self.half = window_size // 2
        self.signal_caches = {}

    def __len__(self):
        return len(self.entries)

    def _get_signal(self, rec_id): #hpf 제거 & HPF 적용
        if rec_id not in self.signal_caches:
            record = wfdb.rdrecord(str(self.base / rec_id))
            sig = record.p_signal[:,0]
            self.signal_caches[rec_id] = highpass_filter(sig)
        return self.signal_caches[rec_id]

    def __getitem__(self, idx): # r_pick기준으로 슬라이싱
        rec_id, sample, label = self.entries[idx]
        sig = self._get_signal(rec_id)

        start = int(sample - self.half)
        end   = int(sample + self.half + 1)

        if start < 0 or end > sig.shape[0]: # seg 길이가 361이 되지 않으면 패딩
            seg = np.zeros(self.window_size, dtype=np.float32)
        else:
            seg = sig[start:end].astype(np.float32) # 슬라이싱 윈도우
            mean, std = seg.mean(), seg.std()# 정규화
            seg = np.zeros_like(seg) if std==0 else (seg-mean)/std

        return torch.from_numpy(seg).float(), torch.tensor(int(label)).long()

# 2.4 AI 모델 학습

Features:
- On-the-fly Dataset that caches per-record filtered signal (HPF)
- CNN-LSTM model (conv->pool->conv->pool->LSTM->FC)
- Weighted CrossEntropyLoss to handle class imbalance
- Optional WeightedRandomSampler alternative (commented)
- Training loop with validation, scheduler, early stopping, and metrics (precision/recall/f1)
- Model saving (best val F1)

In [ ]:
import os
import time
from collections import Counter

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score

HYP = {
    'window_size': 361,
    'batch_size': 64,
    'num_epochs': 60,
    'lr': 1e-3,
    'weight_decay': 1e-5,
    'conv_channels': [32, 64],
    'kernel_size': 5,
    'lstm_hidden': 64,
    'lstm_layers': 1,
    'bidirectional': False,
    'dropout': 0.3,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'patience': 8,  # early stopping patience (epochs)
}

CNNLSTM_ECG — 1D conv 2층 → 풀링 → 시퀀스 차원으로 변환 → LSTM → FC

In [ ]:
# ---------------- Model (CNN -> LSTM) --------------------------------
class CNNLSTM_ECG(nn.Module):
    '''
    in_channels: 입력 채널 수
    n_classes: 최종 분류할 라벨 수
    conv_channels:CNN계층에서 사용할 필터 개수([32,64]-> 첫번째 합성곱은 32개의 특징맵 생성)
    kernel_size: CNN 커널 길이.인접한 n개의 시점 정보를 활용해서 특징을 잡음 (작으면 세밀한 특징, 크면 넓은 패턴을 잘잡음)
    lstm_hidden: 한 시점에서 기억하는 특징의 차원 수
    lstm_layers:LSTM 층 개수
    bidirectional:LSTM을 양방향으로 쓸지 여부(False:과거->현재 일방향)
    dropout: 뉴런 일부를 학습 중 무작위로 꺼서 과적합을 방지하는 비율
    '''

    def __init__(self, in_channels=1, n_classes=2, conv_channels=[32,64], kernel_size=5,
                 lstm_hidden=64, lstm_layers=1, bidirectional=False, dropout=0.3):
        super().__init__()
        padding = kernel_size // 2
        self.conv1 = nn.Conv1d(in_channels, conv_channels[0], kernel_size, padding=padding)
        self.bn1 = nn.BatchNorm1d(conv_channels[0])
        self.pool1 = nn.MaxPool1d(2)

        self.conv2 = nn.Conv1d(conv_channels[0], conv_channels[1], kernel_size, padding=padding)
        self.bn2 = nn.BatchNorm1d(conv_channels[1]) # 1원데이터 배치 정규화
        self.pool2 = nn.MaxPool1d(2) # 1차원입력 최댓값 풀링

        self.relu = nn.ReLU() # 비선형 활성화
        self.dropout = nn.Dropout(dropout)

        self.bidirectional = bidirectional
        self.lstm = nn.LSTM(
            input_size = conv_channels[1],
            hidden_size = lstm_hidden,
            num_layers = lstm_layers,
            batch_first = True,
            bidirectional = bidirectional,
            dropout = dropout if lstm_layers>1 else 0.0
        )

        lstm_out_dim = lstm_hidden * (2 if bidirectional else 1)
        self.fc = nn.Linear(lstm_out_dim, n_classes) # 입력에대해 y - xw^T + b 변환을 수행하는 FC
    def forward(self, x):
        # Input x: (batch, channel=1, length)
        x = self.pool1(self.relu(self.bn1(self.conv1(x))))
        x = self.pool2(self.relu(self.bn2(self.conv2(x))))

        # x: (batch, channels=conv_channels[1], seq_len_reduced)
        x = x.permute(0, 2, 1)  # -> (batch, seq_len, feature)
        out, (h_n, c_n) = self.lstm(x)
        # take last hidden state
        if self.bidirectional:
            h = torch.cat([h_n[-2], h_n[-1]], dim=1)
        else:
            h = h_n[-1]
        h = self.dropout(h)
        return self.fc(h)

In [ ]:
# ---------------- Utilities -------------------------------------------
def compute_class_weights_from_df(df):
    labels = df['label'].to_numpy()
    counts = Counter(labels)
    # weight = total / (num_classes * count) typical
    num_classes = len(counts)
    total = sum(counts.values())
    weights = []
    for cls in range(num_classes):
        weights.append(total / (num_classes * (counts.get(cls,0) + 1e-6)))
    return torch.tensor(weights, dtype=torch.float32)


def collate_fn(batch):
    # batch: list of tuples (segment_tensor (L,), label)
    xs = torch.stack([b[0] for b in batch], dim=0)  # (B, L)
    xs = xs.unsqueeze(1)  # (B, 1, L)
    ys = torch.tensor([int(b[1]) for b in batch], dtype=torch.long)
    return xs, ys

In [ ]:
# ---------------- Training / Evaluation loops -------------------------
def evaluate(model, dataloader, device):
    model.eval()
    ys_all = []
    ps_all = []
    with torch.no_grad():
        for xb, yb in dataloader:
            xb = xb.to(device)
            yb = yb.to(device)
            logits = model(xb)
            probs = torch.softmax(logits, dim=1)[:,1].detach().cpu().numpy()  # positive probs
            preds = logits.argmax(dim=1).detach().cpu().numpy()
            ys_all.extend(yb.detach().cpu().numpy().tolist())
            ps_all.extend(preds.tolist())
    p, r, f1, _ = precision_recall_fscore_support(ys_all, ps_all, average='binary', zero_division=0)
    # roc auc (if both classes present)
    try:
        auc = roc_auc_score(ys_all, np.array(ps_all))
    except Exception:
        auc = float('nan')
    return {'precision':p, 'recall':r, 'f1':f1, 'auc':auc}

In [ ]:
def train_full(train_df, val_df, test_df, base_path, hyp=HYP, save_path='./best_model.pth'):
    device = torch.device(hyp['device'])

    # Datasets
    train_ds = ECGDataset(train_df, base_path, window_size=hyp['window_size'])
    val_ds   = ECGDataset(val_df,   base_path, window_size=hyp['window_size'])
    test_ds  = ECGDataset(test_df,  base_path, window_size=hyp['window_size'])

    train_loader = DataLoader(train_ds, batch_size=hyp['batch_size'], shuffle=True, collate_fn=collate_fn, num_workers=2)
    val_loader   = DataLoader(val_ds,   batch_size=hyp['batch_size'], shuffle=False, collate_fn=collate_fn, num_workers=2)
    test_loader  = DataLoader(test_ds,  batch_size=hyp['batch_size'], shuffle=False, collate_fn=collate_fn, num_workers=2)

    model = CNNLSTM_ECG(
        in_channels=1,
        n_classes=2,
        conv_channels=hyp['conv_channels'],
        kernel_size=hyp['kernel_size'],
        lstm_hidden=hyp['lstm_hidden'],
        lstm_layers=hyp['lstm_layers'],
        bidirectional=hyp['bidirectional'],
        dropout=hyp['dropout']
    ).to(device)

    # Loss with class weights
    class_weights = compute_class_weights_from_df(train_df).to(device)
    criterion = nn.CrossEntropyLoss(weight=class_weights)

    optimizer = torch.optim.Adam(model.parameters(), lr=hyp['lr'], weight_decay=hyp['weight_decay'])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5)

    best_val_f1 = -1.0
    epochs_no_improve = 0

    for epoch in range(1, hyp['num_epochs']+1):
        model.train()
        epoch_loss = 0.0
        n_samples = 0
        t0 = time.time()
        for xb, yb in train_loader:
            xb = xb.to(device)
            yb = yb.to(device)
            optimizer.zero_grad()
            logits = model(xb)
            loss = criterion(logits, yb)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item() * xb.size(0)
            n_samples += xb.size(0)
        epoch_loss /= max(1, n_samples)

        val_metrics = evaluate(model, val_loader, device)
        print(f"Epoch {epoch:03d} | train_loss: {epoch_loss:.4f} | val_f1: {val_metrics['f1']:.4f} | val_prec: {val_metrics['precision']:.4f} | val_rec: {val_metrics['recall']:.4f} | time: {time.time()-t0:.1f}s")

        # scheduler on val loss proxy (use 1 - f1 as loss) or use epoch_loss
        scheduler.step(1.0 - val_metrics['f1'])

        # early stopping logic
        if val_metrics['f1'] > best_val_f1:
            best_val_f1 = val_metrics['f1']
            epochs_no_improve = 0
            torch.save({'model_state': model.state_dict(), 'hyp': hyp}, save_path)
            print(f"  -> new best model saved (val_f1 {best_val_f1:.4f})")
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= hyp['patience']:
                print(f"Early stopping (no improvement for {hyp['patience']} epochs). Best val_f1={best_val_f1:.4f}")
                break

    # load best
    ckpt = torch.load(save_path, map_location=device)
    model.load_state_dict(ckpt['model_state'])
    test_metrics = evaluate(model, test_loader, device)
    print("Test metrics:", test_metrics)
    return model, best_val_f1, test_metrics


In [ ]:
# ---------------- Example entrypoint ---------------------------------
if __name__ == '__main__':
    try:
        model, best_f1, test_metrics = train_full(train_df, val_df, test_df, BASE)
    except Exception as e:
        print("Error running training:", e)
        raise

Epoch 001 | train_loss: 0.3982 | val_f1: 0.1878 | val_prec: 0.1132 | val_rec: 0.5504 | time: 9.7s
  -> new best model saved (val_f1 0.1878)
Epoch 002 | train_loss: 0.2255 | val_f1: 0.2722 | val_prec: 0.1671 | val_rec: 0.7339 | time: 8.7s
  -> new best model saved (val_f1 0.2722)
Epoch 003 | train_loss: 0.1870 | val_f1: 0.4343 | val_prec: 0.2952 | val_rec: 0.8214 | time: 8.5s
  -> new best model saved (val_f1 0.4343)
Epoch 004 | train_loss: 0.1700 | val_f1: 0.4553 | val_prec: 0.3156 | val_rec: 0.8165 | time: 12.9s
  -> new best model saved (val_f1 0.4553)
Epoch 005 | train_loss: 0.1360 | val_f1: 0.3313 | val_prec: 0.2133 | val_rec: 0.7424 | time: 13.2s
Epoch 006 | train_loss: 0.1207 | val_f1: 0.4921 | val_prec: 0.3564 | val_rec: 0.7947 | time: 7.8s
  -> new best model saved (val_f1 0.4921)
Epoch 007 | train_loss: 0.1178 | val_f1: 0.4139 | val_prec: 0.2786 | val_rec: 0.8044 | time: 8.6s
Epoch 008 | train_loss: 0.1170 | val_f1: 0.2855 | val_prec: 0.1706 | val_rec: 0.8736 | time: 8.5s
Epoc

# 2.5 TFLite로 내보내기 & 양자화

In [ ]:
import tensorflow as tf
# TFLite 컨버터를 생성
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# 양자화 옵션을 설정
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# TFLite 모델로 변환
ECG_tflite_model = converter.convert()

# 변환된 모델을 파일로 저장
with open('my_model.tflite', 'wb') as f:
    f.write(ECG_tflite_model)

AttributeError: 'CNNLSTM_ECG' object has no attribute 'call'